In [1]:
domains = ["lunar_habitat_airlock"]

# HRRT Utility Testing

In [2]:
from google.colab import drive
from google.colab import runtime
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Object\ Affordances/HRRT/human_robot_red_teaming
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Object Affordances/HRRT/human_robot_red_teaming
config/  docs/  hrrt.ipynb  README.md  sar/  scripts/


In [3]:
# %cd ../
# %ls
# %cd human_robot_red_teaming
%cd scripts/
%ls

/content/gdrive/MyDrive/Object Affordances/HRRT/human_robot_red_teaming/scripts
hrrt2.py  hrrt3.py  __pycache__/  yaml_formatting_checks.py


## Test Knowledge Base and Model Formatting

In [4]:
from yaml_formatting_checks import KBFormatting, ModelFormatting

kb_file = "../config/lunar_habitat_airlock/knowledge_base.yaml"
model_file = "../config/lunar_habitat_airlock/model0.yaml"

success, kb = KBFormatting.format_kb(kb_file)
if success:
  print("KNOWLEDGE BASE READING SUCCESSFUL!")
else:
  print("Knowledge base reading failed :(")

succes, model = ModelFormatting.format_model(model_file)
if success:
  print("MODEL READING SUCCESSFUL!")
else:
  print("Model reading failed :(")

KNOWLEDGE BASE READING SUCCESSFUL!
MODEL READING SUCCESSFUL!


## Test HRRT Level 2

In [5]:
from hrrt2 import hrrt2, write_hrrt2_yaml

possibilities = hrrt2(model)
print("Found " + str(len(possibilities)) + " possibilities on level HRRT2!")

write_hrrt2_yaml(possibilities, model_file)

print("LEVEL HRRT2 SUCCESSFUL!")

Found 30 possibilities on level HRRT2!
Wrote data for HRRT Level 2 to file: ../config/lunar_habitat_airlock/model0_hrrt2.yaml
LEVEL HRRT2 SUCCESSFUL!


## Test HRRT Level 3

In [6]:
from hrrt3 import hrrt3, write_hrrt3_yaml

assumptions = hrrt3(model)
pre_assump, post_add_assump, post_sub_assump = assumptions
print("Found " + str(len(pre_assump) + len(pre_assump) + len(pre_assump)) + " assumptions on level HRRT3!")

write_hrrt3_yaml(assumptions, model_file)

print("LEVEL HRRT3 SUCCESSFUL!")

Implicit assumption that pre-condition state robot_inside_airlock can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) y
Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) q
Invalid input, please answer [Y/N] for yes or no.
Implicit assumption that pre-condition state robot_has_key can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) n
Implicit assumption that pre-condition state door_habitat_airlock_locked_closed can be achieved to perform action unlock_open_door_habitat_airlock
    Is this assumption always valid? (Y/N) Y
Implicit assumption that pre-condition state robot_outside_habitat can be achieved to perform action unlock_open_door_airlock_surface
    Is this assumption always valid? (Y/N) Y
Implicit assumption that pre-condition state robot_has_key 